# ทดสอบ Face API - คู่มือสำหรับมือใหม่

**สำหรับ Person 3: ผู้ดูแล Face Expression Detection**

## เกี่ยวกับ Notebook นี้

Notebook นี้ใช้สำหรับทดสอบ Face API ก่อนนำไปใช้ใน Frontend:
- ตรวจจับอารมณ์ (Happy, Sad, Angry, Surprised, Neutral, etc.)
- ตรวจจับใบหน้า (Face Detection)
- วัดความแม่นยำ (Confidence Score)

## วัตถุประสงค์

1. **ทดสอบโมเดลก่อนใช้งานจริง** - ตรวจสอบว่าโมเดลทำงานได้ดี
2. **วัดความแม่นยำ** - ดู confidence score ต้องสูงกว่า 50%
3. **ปรับปรุงพารามิเตอร์** - หาค่าที่เหมาะสมสำหรับโปรเจค

## ขั้นตอนการทำงาน

1. ตรวจสอบไฟล์โมเดล Face API
2. โหลดและทดสอบโมเดล  
3. ทดสอบกับรูปภาพตัวอย่าง
4. ปรับปรุงความแม่นยำ
5. เชื่อมต่อกับระบบหลัก

In [ ]:
# ขั้นตอนที่ 1: ติดตั้ง Libraries

print("+ กำลังติดตั้ง libraries สำหรับ Face Detection...")
print("  กรุณารอสักครู่ (อาจใช้เวลา 2-3 นาที)")

# ติดตั้ง libraries ที่จำเป็น
import sys
!{sys.executable} -m pip install opencv-python pillow numpy matplotlib requests
!{sys.executable} -m pip install mediapipe

print("+ ติดตั้งเสร็จเรียบร้อย!")
print("="*50)

In [ ]:
# ขั้นตอนที่ 2: Import Libraries

print("+ กำลัง import libraries...")

# Import libraries ที่จำเป็นทั้งหมด
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import mediapipe as mp

# แสดงข้อมูลระบบ
print("+ Import สำเร็จ!")
print(f"  - OpenCV Version: {cv2.__version__}")
print(f"  - MediaPipe Version: {mp.__version__}")
print(f"  - Current Directory: {os.getcwd()}")
print("="*50)

In [ ]:
# ขั้นตอนที่ 3: ตรวจสอบไฟล์โมเดล Face API (ถ้าใช้ face-api.js)

print("+ ตรวจสอบไฟล์โมเดล Face Detection...")

# ไฟล์โมเดลที่อาจจะมี (สำหรับ face-api.js)
optional_files = [
    'tiny_face_detector_model-weights_manifest.json',
    'face_landmark_68_model-weights_manifest.json',
    'face_expression_model-weights_manifest.json'
]

print("  ผลการตรวจสอบไฟล์โมเดล (ไม่บังคับ):")
found_models = 0

for file in optional_files:
    if os.path.exists(file):
        file_size = os.path.getsize(file) / 1024  # KB
        print(f"  + {file} - พบแล้ว ({file_size:.1f} KB)")
        found_models += 1
    else:
        print(f"  - {file} - ไม่พบ")

print(f"\n+ พบไฟล์โมเดล: {found_models}/{len(optional_files)}")

if found_models > 0:
    print("+ มีไฟล์โมเดล face-api.js สำหรับความแม่นยำสูง")
else:
    print("o ไม่พบไฟล์โมเดล face-api.js - ใช้ MediaPipe แทน (ปกติ)")

print("o หมายเหตุ: Notebook นี้ใช้ MediaPipe (ไม่ต้องใช้ไฟล์เพิ่มเติม)")
print("="*50)

In [ ]:
# ขั้นตอนที่ 4: สร้าง Face Detector Class

print("+ สร้าง Face Detector Class...")

class FaceDetector:
    """
    คลาสง่ายๆ สำหรับตรวจจับใบหน้าและอารมณ์
    """
    
    def __init__(self):
        """เริ่มต้น Face Detector"""
        print("  o กำลังสร้าง Face Detector...")
        self.mp_face_detection = mp.solutions.face_detection
        self.mp_drawing = mp.solutions.drawing_utils
        
        # ตั้งค่า Face Detection
        self.face_detection = self.mp_face_detection.FaceDetection(
            model_selection=0,  # 0 = ใกล้ (<2m), 1 = ไกล (>2m)
            min_detection_confidence=0.5  # ความแม่นยำขั้นต่ำ 50%
        )
        print("  + สร้าง Instance เรียบร้อย!")
        
    def detect_simple(self, image_path):
        """ตรวจจับใบหน้าแบบง่าย"""
        try:
            # อ่านรูปภาพ
            image = cv2.imread(image_path)
            if image is None:
                return {'error': 'ไม่สามารถอ่านไฟล์ได้', 'success': False}
            
            # แปลงเป็น RGB สำหรับ MediaPipe
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # ตรวจจับใบหน้า
            results = self.face_detection.process(rgb_image)
            
            detected_faces = []
            if results.detections:
                for i, detection in enumerate(results.detections):
                    confidence = detection.score[0] * 100
                    detected_faces.append({
                        'face_id': i + 1,
                        'confidence': confidence
                    })
                    
                    # วาดกรอบรอบใบหน้า
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, ic = image.shape
                    x = int(bboxC.xmin * iw)
                    y = int(bboxC.ymin * ih)
                    w = int(bboxC.width * iw)
                    h = int(bboxC.height * ih)
                    
                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(image, f'Face {i+1}: {confidence:.1f}%', 
                               (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            
            return {
                'total_faces': len(detected_faces),
                'faces': detected_faces,
                'image_with_boxes': image,
                'success': True
            }
            
        except Exception as e:
            return {'error': str(e), 'success': False}

print("+ Face Detector Class พร้อมใช้งาน!")
print("="*50)

In [ ]:
# ขั้นตอนที่ 5: สร้าง Face Detector

print("+ เริ่มสร้าง Face Detector...")

# สร้าง instance ของ Face Detector
face_detector = FaceDetector()

print("+ Face Detector พร้อมใช้งาน!")
print("  ฟังก์ชันที่พร้อมใช้:")
print("    - face_detector.detect_simple(image_path)")
print("="*50)

In [ ]:
# ขั้นตอนที่ 6: ทดสอบด้วยรูปภาพ (เพียงภาพเดียว)

print("+ กำลังทดสอบการจดจำหน้า...")

# ทดสอบด้วยรูปตัวอย่าง
test_image = 'test_face.jpg'  # เปลี่ยนเป็นชื่อไฟล์รูปที่ต้องการทดสอบ

if os.path.exists(test_image):
    print(f"  o กำลังตรวจจับหน้าในไฟล์: {test_image}")
    result = face_detector.detect_simple(test_image)
    
    if result['success']:
        print(f"  + ตรวจพบหน้าแล้ว {result['total_faces']} หน้า!")
        
        # แสดงรูปภาพพร้อมกรอบ
        plt.figure(figsize=(10, 6))
        result_rgb = cv2.cvtColor(result['image_with_boxes'], cv2.COLOR_BGR2RGB)
        plt.imshow(result_rgb)
        plt.title(f'ตรวจพบ {result["total_faces"]} หน้า')
        plt.axis('off')
        plt.show()
        
        # แสดงรายละเอียด
        for i, face in enumerate(result['faces']):
            print(f"    หน้าที่ {face['face_id']}: ความมั่นใจ {face['confidence']:.1f}%")
            
    else:
        print("  - ไม่พบหน้าในรูปภาพ")
        if 'error' in result:
            print(f"    ข้อผิดพลาด: {result['error']}")
else:
    print(f"  - ไม่พบไฟล์ {test_image}")
    print("  o กรุณาวางไฟล์รูปในโฟลเดอร์เดียวกับ notebook นี้")
    print("  o หรือเปลี่ยนชื่อไฟล์ในตัวแปร test_image")
    print("  o ไฟล์ที่รองรับ: .jpg, .png, .jpeg")

print("="*50)

In [ ]:
# ขั้นตอนที่ 7: ทดสอบด้วยหลายรูปภาพ

print("+ กำลังทดสอบหลายรูปภาพ...")

# ค้นหาไฟล์รูปทั้งหมดในโฟลเดอร์
import glob
image_files = glob.glob("*.jpg") + glob.glob("*.png") + glob.glob("*.jpeg")

if image_files:
    print(f"  o พบไฟล์รูป {len(image_files)} ไฟล์")
    
    correct_detections = 0
    total_images = 0
    
    # ทดสอบไฟล์รูป (สูงสุด 5 ไฟล์)
    test_files = image_files[:5]
    
    for img_file in test_files:
        print(f"  o ทดสอบ: {img_file}")
        result = face_detector.detect_simple(img_file)
        total_images += 1
        
        if result['success'] and result['total_faces'] > 0:
            correct_detections += 1
            print(f"    + ตรวจพบ {result['total_faces']} หน้า")
        else:
            print(f"    - ไม่พบหน้า")
    
    # แสดงสถิติ
    accuracy = (correct_detections / total_images) * 100 if total_images > 0 else 0
    print(f"\n  สถิติการทดสอบ:")
    print(f"    + ทดสอบทั้งหมด: {total_images} รูป")
    print(f"    + ตรวจพบหน้า: {correct_detections} รูป")
    print(f"    + อัตราความสำเร็จ: {accuracy:.1f}%")
    
    # ประเมินผล
    if accuracy >= 80:
        print("  + ยอดเยี่ยม! ระบบทำงานได้ดี")
    elif accuracy >= 60:
        print("  o ดี ใช้งานได้")
    else:
        print("  - ควรปรับปรุง")
        print("    แนะนำ: ทดสอบด้วยรูปหน้าที่ชัดเจนมากขึ้น")
    
else:
    print("  - ไม่พบไฟล์รูปในโฟลเดอร์นี้")
    print("  o กรุณาวางไฟล์รูป (.jpg, .png, .jpeg) ในโฟลเดอร์เดียวกับ notebook")
    print("  o หรือสร้างไฟล์ทดสอบตัวอย่าง")

print("="*50)

In [ ]:
# ขั้นตอนที่ 8: ทดสอบด้วยกล้องเว็บแคม (Real-time)

print("+ เตรียมทดสอบแบบ Real-time ด้วยกล้อง...")

def start_webcam_face_test():
    """
    เริ่มทดสอบการจดจำหน้าแบบ Real-time
    """
    print("  o กำลังเปิดกล้อง...")
    print("  o กด 'q' เพื่อหยุดการทดสอบ")
    
    # เปิดกล้อง
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("  - ไม่สามารถเปิดกล้องได้")
        return False
    
    frame_count = 0
    detection_count = 0
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            
            # ตรวจจับหน้าในเฟรม
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_detector.face_detection.process(rgb_frame)
            
            if results.detections:
                detection_count += 1
                # วาดกรอบรอบหน้า
                for detection in results.detections:
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, ic = frame.shape
                    x = int(bboxC.xmin * iw)
                    y = int(bboxC.ymin * ih)
                    w = int(bboxC.width * iw)
                    h = int(bboxC.height * ih)
                    
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    
                    confidence = detection.score[0] * 100
                    cv2.putText(frame, f'Face: {confidence:.1f}%', 
                               (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                
                # แสดงสถานะ
                cv2.putText(frame, f"Face Detected! ({detection_count}/{frame_count})", 
                          (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            else:
                cv2.putText(frame, f"No Face ({detection_count}/{frame_count})", 
                          (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            # แสดงเฟรม
            cv2.imshow('Face Detection Test - Press Q to quit', frame)
            
            # ตรวจสอบการกดปุ่ม q
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    except Exception as e:
        print(f"  - เกิดข้อผิดพลาด: {e}")
    finally:
        cap.release()
        cv2.destroyAllWindows()
        
        # แสดงสถิติ
        accuracy = (detection_count / frame_count) * 100 if frame_count > 0 else 0
        print(f"\n  สถิติการทดสอบ Real-time:")
        print(f"    + เฟรมทั้งหมด: {frame_count}")
        print(f"    + ตรวจพบหน้า: {detection_count} เฟรม")
        print(f"    + อัตราการตรวจจับ: {accuracy:.1f}%")
        
    return True

# เรียกใช้ฟังก์ชันทดสอบ (comment ไว้ก่อน เรียกใช้เมื่อพร้อม)
# start_webcam_face_test()

print("  o ฟังก์ชันพร้อมใช้งาน: start_webcam_face_test()")
print("  o ลบ comment หน้าฟังก์ชันเพื่อทดสอบ")
print("="*50)

In [ ]:
# ขั้นตอนที่ 9: ประเมินคุณภาพและประสิทธิภาพ

print("+ กำลังประเมินคุณภาพของ Face Detection...")

import time

def check_face_detection_quality():
    """
    ตรวจสอบคุณภาพการทำงานของระบบ Face Detection
    """
    
    print("  o ทดสอบความเร็วในการประมวลผล...")
    
    # สร้างรูปทดสอบ (สีเดียว)
    test_image = np.ones((480, 640, 3), dtype=np.uint8) * 128
    temp_file = "temp_test.jpg"
    cv2.imwrite(temp_file, test_image)
    
    # ทดสอบความเร็ว
    times = []
    for i in range(10):
        start_time = time.time()
        result = face_detector.detect_simple(temp_file)
        end_time = time.time()
        times.append(end_time - start_time)
        
    # คำนวณสถิติ
    avg_time = np.mean(times)
    min_time = min(times)
    max_time = max(times)
    
    print(f"    + เวลาเฉลี่ย: {avg_time:.3f} วินาที")
    print(f"    + เวลาเร็วสุด: {min_time:.3f} วินาที")
    print(f"    + เวลาช้าสุด: {max_time:.3f} วินาที")
    print(f"    + FPS ประมาณ: {1/avg_time:.1f} frames/sec")
    
    # ลบไฟล์ทดสอบ
    if os.path.exists(temp_file):
        os.remove(temp_file)
    
    # ประเมินประสิทธิภาพ
    print("\n  o การประเมินประสิทธิภาพ:")
    
    if avg_time < 0.1:
        print("    + ประสิทธิภาพ: ดีเยี่ยม (เหมาะกับ Real-time)")
    elif avg_time < 0.2:
        print("    + ประสิทธิภาพ: ดี (ใช้งานได้ปกติ)")
    elif avg_time < 0.5:
        print("    + ประสิทธิภาพ: พอใช้ (อาจช้าในการ Real-time)")
    else:
        print("    - ประสิทธิภาพ: ช้า (ควรปรับปรุง)")
    
    # แนะนำการปรับปรุง
    print("\n  o ข้อแนะนำสำหรับการปรับปรุง:")
    print("    + ลดขนาดรูปภาพก่อนประมวลผล")
    print("    + ใช้ confidence threshold ที่เหมาะสม")
    print("    + พิจารณาใช้ GPU หากต้องการความเร็วสูง")
    
    return {
        'avg_time': avg_time,
        'fps': 1/avg_time,
        'performance': 'ดีเยี่ยม' if avg_time < 0.1 else 'ดี' if avg_time < 0.2 else 'พอใช้' if avg_time < 0.5 else 'ช้า'
    }

# เรียกใช้การประเมิน
performance = check_face_detection_quality()

print("="*50)

In [ ]:
# ขั้นตอนที่ 10: สรุปและวิธีใช้งาน

print("+ สรุป Face Detection API Test")
print("="*50)

print("\nฟังก์ชันที่พร้อมใช้งาน:")
print("+ face_detector.detect_simple(image_path)")
print("  - ตรวจจับหน้าในรูปภาพ")
print("  - แสดงผลด้วย matplotlib")

print("\n+ start_webcam_face_test()")
print("  - ทดสอบแบบ Real-time ด้วยกล้อง")
print("  - กด 'q' เพื่อหยุด")

print("\n+ check_face_detection_quality()")
print("  - ประเมินประสิทธิภาพการทำงาน")
print("  - วัดความเร็วในการประมวลผล")

print("\nวิธีใช้งานสำหรับมือใหม่:")
print("o รันทุกเซลล์จากข้างบนตามลำดับ")
print("o ใส่ไฟล์รูปใน folder เดียวกันกับ notebook นี้")
print("o แก้ไขชื่อไฟล์ในตัวแปร test_image หากต้องการ")
print("o สำหรับ Real-time test ให้ลบ comment หน้า start_webcam_face_test()")

print("\nข้อมูลเพิ่มเติม:")
print("- Face Detection ใช้ MediaPipe")
print("- รองรับไฟล์ .jpg และ .png")
print("- เหมาะสำหรับการจดจำหน้าในโปรเจค Thai-HandMate")

print("\n+ การทดสอบ Face Detection เสร็จสิ้น!")
print("+ พร้อมใช้งานในระบบหลักได้แล้ว")
print("="*50)